In [1]:
"""
1.651.119 -> Στο σύνολο γραμμές καταχωρημένων ενδείξεων.
752.146 -> Ομαδοποιημένες εγγραφές στο 8ώρο.
1.547 -> Eισαγωγές στις ΜΕΘ
1.662 -> Ομαδοποιημένες εγγραφές στο 8ώρο που αφορούν ασθενείς από τα επείγοντα στις ΜΕΘ
"""

'\n1.651.119 -> Στο σύνολο γραμμές καταχωρημένων ενδείξεων.\n752.146 -> Ομαδοποιημένες εγγραφές στο 8ώρο.\n1.547 -> Eισαγωγές στις ΜΕΘ\n1.662 -> Ομαδοποιημένες εγγραφές στο 8ώρο που αφορούν ασθενείς από τα επείγοντα στις ΜΕΘ\n'

In [2]:
import pandas as pd

In [3]:
df_vitalsign = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\ed\Vitalsign.csv.gz')
df_ed_to_icu = pd.read_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o16_ed_patients_to_icu.csv', usecols = ['subject_id', 'hadm_id','stay_id'])

In [4]:
# Μετατροπή της στήλης charttime από object σε datetime
df_vitalsign['charttime']= pd.to_datetime(df_vitalsign['charttime'])

In [5]:
# Ομαδοποίηση καταγραφών ανά στήλη στο 8ώρο.

grouped_multiple = (df_vitalsign.groupby(['subject_id', 'stay_id',
                                          pd.Grouper(key='charttime',
                                                     freq='8H')]).agg({'temperature': ['mean'],
                                                                       'heartrate': ['mean'],
                                                                       'resprate': ['mean'],
                                                                       'o2sat': ['mean'],
                                                                       'sbp': ['mean'],
                                                                       'dbp': ['mean']}))
grouped_multiple.columns = (['Mean_temperature',
                             'Mean_heartrate',
                             'Mean_resprate',
                             'Mean_o2sat',
                             'Mean_sbp',
                             'Mean_dbp'])
grouped_multiple = grouped_multiple.reset_index()

In [6]:
# Κενές τιμές στον πίνακα
grouped_multiple.isna().sum()

subject_id              0
stay_id                 0
charttime               0
Mean_temperature    60071
Mean_heartrate       6126
Mean_resprate        7643
Mean_o2sat          28566
Mean_sbp             7394
Mean_dbp             7394
dtype: int64

In [7]:
df_ed_to_icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   subject_id  1547 non-null   int64
 1   hadm_id     1547 non-null   int64
 2   stay_id     1547 non-null   int64
dtypes: int64(3)
memory usage: 36.4 KB


In [8]:
rslt_df=pd.merge(df_ed_to_icu, grouped_multiple, on=['stay_id'])

In [9]:
rslt_df = rslt_df.drop(['subject_id_y'], axis=1)

In [10]:
rslt_df = rslt_df.rename(columns={"subject_id_x": "subject_id"})

In [11]:
rslt_df.insert(0, 'Department', 'ed')

In [12]:
rslt_df

,Department,subject_id,hadm_id,stay_id,charttime,Mean_temperature,Mean_heartrate,Mean_resprate,Mean_o2sat,Mean_sbp,Mean_dbp
0,ed,13397589,22849351,33940471,2186-10-03 16:00:00,98.90,71.750000,15.250000,98.333333,164.500000,89.000000
1,ed,15902493,21900706,32961247,2174-07-24 08:00:00,NaN,72.000000,18.000000,99.000000,152.000000,73.000000
2,ed,15902493,21900706,32961247,2174-07-24 16:00:00,98.00,73.500000,17.000000,100.000000,118.000000,59.500000
3,ed,18716059,27648116,37693028,2169-05-08 08:00:00,NaN,66.000000,16.500000,100.000000,171.500000,68.000000
4,ed,18716059,27648116,37693028,2169-05-08 16:00:00,97.90,64.800000,17.600000,100.000000,147.800000,61.200000
...,...,...,...,...,...,...,...,...,...,...,...
1657,ed,16178321,20904950,34164113,2161-04-07 16:00:00,97.60,83.214286,16.714286,95.923077,146.642857,73.071429
1658,ed,16178321,20904950,34164113,2161-04-08 00:00:00,NaN,78.500000,17.500000,97.000000,110.000000,55.500000
1659,ed,13105583,21899375,35230349,2178-10-01 00:00:00,98.00,76.000000,13.666667,99.000000,124.333333,81.666667
1660,ed,13293337,23590234,36006719,2144-04-13 08:00:00,98.45,81.800000,18.600000,98.200000,130.000000,78.500000


In [13]:
#Εξαγωγή σε csv
rslt_df.to_csv('D:\MSc Aegean\Διατριβή\MIMIC_IV\mimic-iv-1.0\exports\o17_ed_to_icu_vitalsigns.csv', index=False)